<a href="https://colab.research.google.com/github/AshishGusain17/via_google_colab/blob/master/snapshotEnsembling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# example of saving models for a snapshot ensemble
from sklearn.datasets import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras import backend
from math import pi
from math import cos
from math import floor





from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np


import matplotlib
matplotlib.use("Agg")

Using TensorFlow backend.


In [2]:
# snapshot ensemble with custom learning rate schedule
class SnapshotEnsemble(Callback):
	# constructor
	def __init__(self, n_epochs, n_cycles, lrate_max, verbose=0):
		self.epochs = n_epochs
		self.cycles = n_cycles
		self.lr_max = lrate_max
		self.lrates = list()

	# calculate learning rate for epoch
	def cosine_annealing(self, epoch, n_epochs, n_cycles, lrate_max):
		epochs_per_cycle = floor(n_epochs/n_cycles)
		cos_inner = (pi * (epoch % epochs_per_cycle)) / (epochs_per_cycle)
		return lrate_max/2 * (cos(cos_inner) + 1)

	# calculate and set learning rate at the start of the epoch
	def on_epoch_begin(self, epoch, logs={}):
		# calculate learning rate
		lr = self.cosine_annealing(epoch, self.epochs, self.cycles, self.lr_max)
		# set learning rate
		backend.set_value(self.model.optimizer.lr, lr)
		# log value
		self.lrates.append(lr)

	# save models at the end of each cycle
	def on_epoch_end(self, epoch, logs={}):
		# check if we can save model
		epochs_per_cycle = floor(self.epochs / self.cycles)
		if epoch != 0 and (epoch + 1) % epochs_per_cycle == 0:
			# save model to file
			filename = "snapshot_model_%d.h5" % int((epoch + 1) / epochs_per_cycle)
			self.model.save(filename)
			print('>saved snapshot %s, epoch %d' % (filename, epoch))


In [3]:

# load the training and testing data, converting the images from integers to floats
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")
print(trainX.shape , testX.shape)

# apply mean subtraction to the data
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)
print(trainY.shape , testY.shape)

# construct the image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.1,
	height_shift_range=0.1, horizontal_flip=True,
	fill_mode="nearest")



[INFO] loading CIFAR-10 data...
170500096/170498071 [==============================] - 6s 0us/step
(50000, 32, 32, 3) (10000, 32, 32, 3)
(50000, 10) (10000, 10)


In [7]:
ct = 0
import keras



m1 = keras.applications.ResNet50(include_top=False,
                  input_shape = (32,32,3),
                  weights = 'imagenet')

model = keras.models.Sequential() 
model.add(m1)

for layer in model.layers:
  ct=ct+1

for layer in model.layers:
  if ct>10:
    layer.trainable = False
  else:
    layer.trainable = True
  ct=ct-1


model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [8]:
model.compile(loss="categorical_crossentropy", 
              optimizer='adam',
	          metrics=["accuracy"])





In [9]:
steps_per_epoch = 512         # number of steps in each epoch
n_epochs = 10                 # number of epochs to be trained
n_cycles = 5                  # number of models to be generated


snapShotEN = SnapshotEnsemble(n_epochs, n_cycles, 0.01)

H = model.fit(
	x=aug.flow(trainX, trainY, 32),
	validation_data=(testX, testY),
	steps_per_epoch=steps_per_epoch,
	epochs=n_epochs,
	callbacks=[snapShotEN],
	verbose=1)



Epoch 1/10
512/512 [==============================] - 147s 288ms/step - loss: 4.4811 - accuracy: 0.1478 - val_loss: 3.9483 - val_accuracy: 0.1789
Epoch 2/10
512/512 [==============================] - 125s 244ms/step - loss: 2.2107 - accuracy: 0.1974 - val_loss: 2.1224 - val_accuracy: 0.1998
>saved snapshot snapshot_model_1.h5, epoch 1
Epoch 3/10
512/512 [==============================] - 125s 244ms/step - loss: 2.1049 - accuracy: 0.2101 - val_loss: 2.4082 - val_accuracy: 0.2143
Epoch 4/10
512/512 [==============================] - 124s 241ms/step - loss: 1.8841 - accuracy: 0.2782 - val_loss: 1.7658 - val_accuracy: 0.3223
>saved snapshot snapshot_model_2.h5, epoch 3
Epoch 5/10
512/512 [==============================] - 125s 245ms/step - loss: 1.8150 - accuracy: 0.3154 - val_loss: 1.8171 - val_accuracy: 0.3392
Epoch 6/10
512/512 [==============================] - 125s 245ms/step - loss: 1.6775 - accuracy: 0.3665 - val_loss: 1.5431 - val_accuracy: 0.4204
>saved snapshot snapshot_model_3.h

In [12]:
new_model5 = keras.models.load_model("/content/snapshot_model_5.h5")
new_model5.evaluate(testX,testY)

10000/10000 [==============================] - 12s 1ms/step


[1.3325670169830321, 0.5156000256538391]

In [13]:
new_model3 = keras.models.load_model("/content/snapshot_model_3.h5")
new_model3.evaluate(testX,testY)

10000/10000 [==============================] - 12s 1ms/step


[1.5431362684249879, 0.4203999936580658]